# Calcul de valeurs propres de Maxwell dans un carré
$\mathrm{curl\, curl}\, u = \lambda u$ dans le carré $(0,\pi) \times\ (0,\pi)$
avec condition limite magnétique homogène $u\times n =0$
Cf Arnold FEEC chap 1


In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from mpi4py import MPI

In [3]:
import dolfinx
print(f"DOLFINx version: {dolfinx.__version__} based on GIT commit: {dolfinx.git_commit_hash} of https://github.com/FEniCS/dolfinx/")

DOLFINx version: 0.7.3 based on GIT commit: ubuntu of https://github.com/FEniCS/dolfinx/


In [4]:
from dolfinx import mesh
from ufl import TrialFunction, TestFunction, grad, dx, dot
from slepc4py import SLEPc
from petsc4py import PETSc
from petsc4py.PETSc import ScalarType
from dolfinx.fem import (Constant, dirichletbc, Function, FunctionSpace, locate_dofs_geometrical,
                         locate_dofs_topological) 
from dolfinx.mesh import CellType,DiagonalType, create_rectangle, create_unit_square, locate_entities_boundary
from ufl import (FacetNormal, FiniteElement, Identity, Measure, TestFunction, TrialFunction, VectorElement,
                 as_vector, div, dot, ds, dx, inner)


In [5]:
#maillage uniforme diagonal
domain = mesh.create_rectangle(MPI.COMM_WORLD,[np.array([0.0, 0.0]), np.array([np.pi, np.pi])], [40, 40], cell_type=CellType.triangle,
                               diagonal=DiagonalType.right) 
#maillage uniforme crisscross
#domain = mesh.create_rectangle(MPI.COMM_WORLD,[np.array([0.0, 0.0]), np.array([np.pi, np.pi])], [40, 40], cell_type=CellType.triangle,
#                               diagonal=DiagonalType.crossed)                

In [6]:
domain.ufl_cell()

triangle

In [7]:
tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(fdim, tdim)
#facets = mesh.exterior_facet_indices(domain.topology)


In [8]:
import ufl
#element = create_element(basix.ElementFamily.N1E, basix.CellType.triangle, 1)
#element = ufl.FiniteElement("Nedelec 1st kind H(curl)", domain.ufl_cell(), 1)
element = FiniteElement("N1curl", domain.ufl_cell(), 1)
#element = VectorElement("CG", domain.ufl_cell(), 1, tdim)
V = FunctionSpace(domain, element)

u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
#u = TrialFunction(V)
#v = TestFunction(V)

In [9]:
def Rot(v):
    return v[1].dx(0) - v[0].dx(1)

In [10]:
a= ufl.dot(ufl.curl(u), ufl.curl(v))*ufl.dx
#a= Rot(u)*Rot(v)*ufl.dx
b = ufl.dot(u,v)*ufl.dx
from dolfinx import fem
bilinear_form = fem.form(a)
mass_matrix = fem.form(b)

In [11]:
print(f"Trial function shape {u.ufl_shape}")
zero = fem.Function(V)
boundary_facets = mesh.exterior_facet_indices(domain.topology)
boundary_dofs = fem.locate_dofs_topological(V, fdim, boundary_facets)
bc = dirichletbc(zero, boundary_dofs)

Trial function shape (2,)


In [12]:
#cellule inutile car Nedelec prescrit automatiquement
#la composante tangentielle
def hori(x):
    return np.logical_or( np.isclose(x[1], 0), np.isclose(x[1],np.pi))
def vert(x):
    return np.logical_or( np.isclose(x[0], 0), np.isclose(x[0],np.pi))

hori_facets = locate_entities_boundary(domain, fdim, hori)
vert_facets = locate_entities_boundary(domain, fdim, vert)
boundary_dofs_x = locate_dofs_topological(V, fdim, hori_facets)
boundary_dofs_y = locate_dofs_topological(V, fdim, vert_facets)
bcx = dirichletbc(zero, boundary_dofs_x)                     
bcy = dirichletbc(zero, boundary_dofs_y) 
# uxn =0 composante tangentielle nulle
bct = [bcx, bcy]
#en fait c'est inutile car l'élément de Nedelec prescrit
#automatiquement la composante tangentielle


In [13]:
import dolfinx.fem.petsc
# Assemble stiffness tensor and mass matrix
A = fem.petsc.assemble_matrix(bilinear_form, bcs=[bc])
#A = fem.petsc.assemble_matrix(bilinear_form, bcs=bct)
# pour éviter des modes propres parasites pour vp 1
# on met 0 sur les noeuds correspondant aux conditions limites
# cela vient de la façon dont son codées les conditions limites
B = fem.petsc.assemble_matrix(mass_matrix,bcs=[bc],diagonal=0.0)
#B = fem.petsc.assemble_matrix(mass_matrix,bcs=bct,diagonal=0.0)
A.assemble()
B.assemble()


In [14]:
# Create SLEPc Eigenvalue solver
eps = SLEPc.EPS().create(PETSc.COMM_WORLD)
eps.setOperators(A, B)
eps.setType(SLEPc.EPS.Type.KRYLOVSCHUR)
eps.setProblemType(SLEPc.EPS.ProblemType.GHEP)
eps.setWhichEigenpairs(eps.Which.TARGET_MAGNITUDE)
#shift 
shift =5.5
eps.setTarget(shift)

st = eps.getST()
st.setType(SLEPc.ST.Type.SINVERT)
st.setShift(shift)
#number of eigenvalues
n_eigs=12
eps.setDimensions(n_eigs, PETSc.DECIDE, PETSc.DECIDE)
eps.setFromOptions()
eps.solve()

its = eps.getIterationNumber()
eps_type = eps.getType()
n_ev, n_cv, mpd = eps.getDimensions()
tol, max_it = eps.getTolerances()
n_conv = eps.getConverged()

print(f"Number of iterations: {its}")
print(f"Solution method: {eps_type}")
print(f"Number of requested eigenvalues: {n_ev}")
print(f"Stopping condition: tol={tol}, maxit={max_it}")
print(f"Number of converged eigenpairs: {n_conv}")

computed_eigenvalues = []
for i in range(min(n_conv, n_eigs)):
    lmbda = eps.getEigenvalue(i)
    computed_eigenvalues.append(np.round(np.real(lmbda), 1))
np.sort(computed_eigenvalues)

Number of iterations: 2
Solution method: krylovschur
Number of requested eigenvalues: 12
Stopping condition: tol=1e-08, maxit=361
Number of converged eigenpairs: 16


array([ 1.,  1.,  2.,  4.,  4.,  5.,  5.,  8.,  9.,  9., 10., 10.])

### On retrouve  exactement les résultats de Arnold, Finite element exterior calculus, SIAM 2018, p9 

### les valeurs propres exactes
$\nabla \times \nabla  u = \lambda u$ sur $(0,\pi) \times (0,\pi)$ avec condition limite magnétique $u \times n =0$
sont données par $u(x,y) =  \mathrm{curl}(\cos(k x) \cos(l y))$ 
et 
$\lambda = (k^2 + l^2)$ pour $k,l \geq 0$ excepté $k=l=0$

In [15]:
list = []
for k in range(0,7):
    for l in range(0,7):
        lamb = (k**2 + l**2)
        list.append(lamb)
list.sort()
print(list[1:])

[1, 1, 2, 4, 4, 5, 5, 8, 9, 9, 10, 10, 13, 13, 16, 16, 17, 17, 18, 20, 20, 25, 25, 25, 25, 26, 26, 29, 29, 32, 34, 34, 36, 36, 37, 37, 40, 40, 41, 41, 45, 45, 50, 52, 52, 61, 61, 72]


In [16]:
import pyvista
print(pyvista.global_theme.jupyter_backend)


trame


In [17]:
#pyvista.set_jupyter_backend('trame')
from dolfinx import plot
tdim = domain.topology.dim
pyvista.start_xvfb()
topology, cell_types, geometry = plot.vtk_mesh(domain, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)


In [18]:
plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("fundamentals_mesh.png")


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.
